In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

### 0. Загрузка данных

In [2]:
!pip install mne
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
!wget https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv

--2025-12-15 00:42:59--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5381138 (5.1M) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>]   5.13M  --.-KB/s    in 0.08s   

2025-12-15 00:43:00 (67.6 MB/s) - ‘train.csv.1’ saved [5381138/5381138]

--2025-12-15 00:43:00--  https://raw.githubusercontent.com/adasegroup/NEUROML2020/seminar1/seminar1/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3567877 (3.4M

In [3]:
!pip install lightautoml

In [47]:
import pandas as pd
import numpy as np
import mne

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, f1_score

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

# from src.dataset import get_target, calc_features

delimiter = "\n" + "-"*10

In [5]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,time,condition,epoch,C3,Cz,C4,Fp1,Fp2,F7,F3,...,F8,T7,T8,P7,P3,Pz,P4,P8,O1,O2
0,0,1,0,-5.885714,-2.533107,9.866895,7.962973,5.694433,23.638605,27.899784,...,23.588723,12.178548,0.685809,-4.887397,-10.646985,-14.735646,-8.729323,-0.562578,-17.055458,-3.616732
1,1,1,0,-7.999715,-16.916729,-11.924855,17.955477,8.526994,56.635981,28.508435,...,1.045533,14.656061,-4.119778,-4.632381,-17.980657,-23.456960,-12.960684,-9.639784,-20.233549,-1.229811
2,2,1,0,-6.727283,-15.979567,-11.114195,17.183478,4.497028,43.914130,10.079754,...,-14.741630,14.793562,-6.624813,-3.402757,-10.269473,-18.736144,-3.579046,-0.045658,-14.089755,2.453398
3,3,1,0,6.819390,-0.204905,10.090124,20.265222,7.843006,36.250611,13.291199,...,-7.135541,21.723418,-2.276825,2.066859,4.325365,-2.803322,8.835114,13.878945,-5.772410,7.657873
4,4,1,0,13.129486,-5.817193,5.040633,19.462210,9.634234,42.311729,20.641012,...,0.015602,19.703190,3.739076,2.714350,3.251047,-3.631448,3.212956,4.635574,-5.045448,5.086024


In [11]:
non_eeg = ['time', 'condition', 'epoch', 'target']
channels = [ch for ch in df_train.columns if ch not in non_eeg]

In [17]:
X = get_target(df_train)
X = X.merge(calc_features(df_train, channels), on='epoch')
y = X['condition'].apply(lambda x: 0 if x == 1 else 1)

X = X.drop(['epoch', 'condition'], axis=1)
y = y.astype(int)
y.name = 'target'
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

## 3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных

[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения

[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

In [18]:
# Cтратегии разделения данных:
# В соответствием с выводами из файла EDA (part 1) делаем корректный train-test split по данным, предварительно подготовленным по эпохам
npr = np.random.RandomState(42)
epoch_list = X.index
train_idx, test_idx = train_test_split(epoch_list, test_size=0.33, random_state=42) ### делаем разбиение по эпохам как независимым объектам, чтобы избежать утечки между зависимыми наблюдениями, поскольку эпохи коррелированы внутри себя

X_train = X.iloc[train_idx]
X_test = X.iloc[test_idx]
y_train = y.iloc[train_idx]
y_test = y.iloc[test_idx]


In [29]:
# Подготовим данные для конфигураций
roles = {'target': 'target'}
task = Task('binary')

train_LAMA = X_train.copy()
train_LAMA['target'] = y_train.values

test_LAMA = X_test.copy()
test_LAMA['target'] = y_test.values

#### Конфигурация 1 - Классический LAMA (default)
Стандартная конфигурация - подбор лучших моделей (LGB, CatBoost и др.) с автотюнингом. Используем пресеты "по умолчанию".

In [25]:
automl1 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
)
oof_pred1 = automl1.fit_predict(train_LAMA, roles=roles)
test_pred1 = automl1.predict(test_LAMA)

auc1 = roc_auc_score(y_test, test_pred1.data[:, 0])
f1_1 = f1_score(y_test, test_pred1.data[:, 0] > 0.5)
print(f"[LAMA Default] ROC-AUC: {auc1:.4f} F1: {f1_1:.4f}")

INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: binary

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 2 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (120, 20)

INFO3:lightautoml.reader.base:Feats was rejected during automatic roles guess: []
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 587.12 secs
INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
DEBUG:lightautoml.ml_algo.base:Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [0, 1], 'embed_sizes': array([11, 11], dtype=int32), 'data_size': 27}
INFO2:lightauto

[LAMA Default] ROC-AUC: 0.7217 F1: 0.6557


### Конфигурация 2 — FAST, только lgbm
Используем только быстрый LightGBM.

In [32]:
automl2 = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb']]
    }
)
oof_pred2 = automl2.fit_predict(train_LAMA, roles=roles)
test_pred2 = automl2.predict(test_LAMA)

auc2 = roc_auc_score(y_test, test_pred2.data[:, 0])
f1_2 = f1_score(y_test, test_pred2.data[:, 0] > 0.5)
print(f'[LAMA FAST (lgb only)] ROC-AUC: {auc2:.4f}  F1: {f1_2:.4f}')


INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: binary

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 300.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 2 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (120, 20)

INFO3:lightautoml.reader.base:Feats was rejected during automatic roles guess: []
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 284.82 secs
INFO3:lightautoml.ml_algo.boost_lgbm:Training until validation scores don't improve for 200 rounds
INFO3:lightautoml.ml_algo.boost_lgbm:[100]	valid's auc: 0.520833
INFO3:lightautoml.ml_algo.boost_lgbm:[200]	valid's auc: 0.534722
INFO3:lightautoml.ml_algo.boost_lgbm:Early stopping, best iteration is:
[51]	valid's auc: 0.569444
INFO:lightautoml.ml_algo.base:Selector_LightGBM fitting and predicting completed
INFO

[LAMA FAST (lgb only)] ROC-AUC: 0.7138  F1: 0.6552


### Конфигурация 3 - расширенный набор моделей и мощный автотюнинг

Включаем сразу несколько ансамблей (XGB, LGBM с тюнингом, CatBoost с тюнингом, MLP нейросеть) + используем продвинутую кросс-валидацию, автотюнинг с разными алгоритмами

In [33]:
automl3 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['xgb', 'lgb_tuned', 'cb_tuned'], ['mlp']],  # Много моделей
    },
    reader_params={
        'n_jobs': 2,
        'cv': 5,
        'random_state': 84,
        'advanced_roles': True,
    },
    tuning_params={'max_tuning_iter': "auto", 'max_tuning_time': 300},
    selection_params={'mode': 1}
)

oof_pred3 = automl3.fit_predict(train_LAMA, roles=roles)
test_pred3 = automl3.predict(test_LAMA)

auc3 = roc_auc_score(y_test, test_pred3.data[:, 0])
f1_3 = f1_score(y_test, test_pred3.data[:, 0] > 0.5)
print(f'[LAMA Advanced] ROC-AUC: {auc3:.4f}  F1: {f1_3:.4f}')


INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: binary

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 2 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (120, 20)

INFO3:lightautoml.reader.base:Feats was rejected during automatic roles guess: []
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 599.62 secs
INFO3:lightautoml.ml_algo.boost_lgbm:Training until validation scores don't improve for 200 rounds
INFO3:lightautoml.ml_algo.boost_lgbm:[100]	valid's auc: 0.784722
INFO3:lightautoml.ml_algo.boost_lgbm:[200]	valid's auc: 0.743056
INFO3:lightautoml.ml_algo.boost_lgbm:[300]	valid's auc: 0.743056
INFO3:lightautoml.ml_algo.boost_lgbm:Early stopping, best iteration is:
[100]	valid's auc: 0.784722
INFO:lightautoml.ml

[LAMA Advanced] ROC-AUC: 0.5000  F1: 0.0000


### конфигурация 4 - с feature selection и ограниченным набором моделей
Только LGBM (c hyperparameter tuning) и линейная регрессия; включён feature selection (оставляем 80% лучших признаков); гиперпараметры подбираются на holdout-валидации.


In [34]:
automl4 = TabularAutoML(
    task=task,
    timeout=600,
    cpu_limit=2,
    general_params={
        'use_algos': [['lgb_tuned', 'linear_l2']]  # Только LGBM + Linear
    },
    selection_params={
        'select_algos': ['gbm', 'linear_l2'],
        'selection_feats_rate': 0.8,
        'max_features_cnt_in_result': None,
    },
    tuning_params={
        'fit_on_holdout': True,
        'max_tuning_iter': 50,
        'max_tuning_time': 200,
    }
)

oof_pred4 = automl4.fit_predict(train_LAMA, roles=roles)
test_pred4 = automl4.predict(test_LAMA)

auc4 = roc_auc_score(y_test, test_pred4.data[:, 0])
f1_4 = f1_score(y_test, test_pred4.data[:, 0] > 0.5)
print(f'[LAMA FS+LGBM/Linear] ROC-AUC: {auc4:.4f}  F1: {f1_4:.4f}')


INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: binary

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 2 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (120, 20)

INFO3:lightautoml.reader.base:Feats was rejected during automatic roles guess: []
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 599.33 secs
INFO3:lightautoml.ml_algo.boost_lgbm:Training until validation scores don't improve for 200 rounds
INFO3:lightautoml.ml_algo.boost_lgbm:[100]	valid's auc: 0.520833
INFO3:lightautoml.ml_algo.boost_lgbm:[200]	valid's auc: 0.534722
INFO3:lightautoml.ml_algo.boost_lgbm:Early stopping, best iteration is:
[51]	valid's auc: 0.569444
INFO:lightautoml.ml_algo.base:Selector_LightGBM fitting and predicting completed
INFO

[LAMA FS+LGBM/Linear] ROC-AUC: 0.7217  F1: 0.4082


#### Выбор лучшего

In [36]:
results = {}
results['LAMA_DEFAULT'] = (auc1, f1_1)
results['LAMA_FAST'] = (auc2, f1_2)
results['LAMA_ADVANCED'] = (auc3, f1_3)
results['LAMA_FS'] = (auc4, f1_4)
for k, v in results.items():
    print(f'{k:15}| ROC-AUC: {v[0]:.4f} | F1: {v[1]:.4f}')

best_lama_name = max(results, key=lambda k: results[k][0])
print(f'--> Лучший бэйзлайн: {best_lama_name}, ROC-AUC = {results[best_lama_name][0]:.4f}')

LAMA_DEFAULT   | ROC-AUC: 0.7217 | F1: 0.6557
LAMA_FAST      | ROC-AUC: 0.7138 | F1: 0.6552
LAMA_ADVANCED  | ROC-AUC: 0.5000 | F1: 0.0000
LAMA_FS        | ROC-AUC: 0.7217 | F1: 0.4082
--> Лучший бэйзлайн: LAMA_DEFAULT, ROC-AUC = 0.7217


Классическая конфигурация LAMA, несмотря на консерватизм, дала лучший результат среди тестируемых.

### Собственное решение 1

In [43]:
# PIPELINE 1: LightGBM + scale + auto feature selection

# PIPELINE 1A: только scaling + LGBM (без фичер селекции)
pipe_lgbm_base = Pipeline([
    ('scaler', StandardScaler()),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_base.fit(X_train, y_train)
pred_lgbm_base = pipe_lgbm_base.predict_proba(X_test)[:, 1]
auc_lgbm_base = roc_auc_score(y_test, pred_lgbm_base)
print(f'[Pipeline LGBM basic] ROC-AUC: {auc_lgbm_base:.4f}')

# PIPELINE 1B: scaling + feature selection + LGBM
# Селектор тренируется внутри пайплайна по cross-val на train (без data leakage)
pipe_lgbm_fs = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(LGBMClassifier(n_estimators=100, random_state=42), threshold="median")),
    ('lgbm', LGBMClassifier(n_estimators=100, random_state=42))
])
pipe_lgbm_fs.fit(X_train, y_train)
pred_lgbm_fs = pipe_lgbm_fs.predict_proba(X_test)[:, 1]
auc_lgbm_fs = roc_auc_score(y_test, pred_lgbm_fs)
print(f'[Pipeline LGBM + feature selection] ROC-AUC: {auc_lgbm_fs:.4f}')

[Pipeline LGBM basic] ROC-AUC: 0.6980
[Pipeline LGBM + feature selection] ROC-AUC: 0.6742


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SelectFromModel was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SelectFromModel was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


### Собственное решение 2

In [45]:
# PIPELINE 2: CatBoost + простая обработка пропусков
pipe_catboost = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('catboost', CatBoostClassifier(iterations=200,
                                    learning_rate=0.05,
                                    eval_metric='AUC',
                                    random_state=42,
                                    verbose=0))
])

pipe_catboost.fit(X_train, y_train)
pred_cat = pipe_catboost.predict_proba(X_test)[:, 1]
auc_cat = roc_auc_score(y_test, pred_cat)
print(f'[Pipeline CatBoost] ROC-AUC: {auc_cat:.4f}')


[Pipeline CatBoost] ROC-AUC: 0.7048


### Собственное решение 3

In [46]:
# PIPELINE 3: Logistic Regression + StandardScaler + гиперпараметры GridSearch
pipe_logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=300))
])

param_grid = {'logreg__C': [0.01, 0.1, 1, 3, 10]}
grid = GridSearchCV(pipe_logreg, param_grid, scoring='roc_auc', cv=5)
grid.fit(X_train, y_train)

print(f"[Pipeline LogReg] Лучшее C: {grid.best_params_}")
best_logreg = grid.best_estimator_
preds_lr = best_logreg.predict_proba(X_test)[:, 1]
auc_lr = roc_auc_score(y_test, preds_lr)
print(f'[Pipeline LogReg tuned] ROC-AUC: {auc_lr:.4f}')

[Pipeline LogReg] Лучшее C: {'logreg__C': 0.1}
[Pipeline LogReg tuned] ROC-AUC: 0.7455


### Вывод
Свое решение оказалось лучше LAMA baseline: максимум ROC-AUC своего решения: 0.7455 (пайплайн 3), лучшего baseline: 0.7217
